In [2]:
import dgl
import torch
import numpy as np
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), '..', ''))
sys.path.append(project_path)

from python.python_code.data_manip.extraction.telemac_file import TelemacFile
from python.create_dgl_dataset import add_mesh_info

#mesh_list_fine = ['/work/m24046/m24046mrcr/results_data_30min/maillage_3.slf']


In [10]:
mesh_list = ['/work/m24046/m24046mrcr/results_data_30min_35_70_maillagex8/Mesh8_corrige.slf']
res_mesh = TelemacFile(mesh_list[0])
X,triangles = add_mesh_info(res_mesh)

In [11]:
graphs, _ = dgl.load_graphs('/work/m24046/m24046mrcr/results_data_30min_35_70_maillagex8/Multimesh_8_32.bin') 

In [67]:
graphs_2, _ = dgl.load_graphs('/work/m24046/m24046mrcr/results_data_30min_35_70_maillagex8/Mesh8_base.bin') 

In [70]:
graphs_3, _ = dgl.load_graphs('/work/m24046/m24046mrcr/results_data_30min/Multimesh_2_32_True.bin')

In [71]:
graph = graphs[0]
graph_2 = graphs_2[0]
graph_3 = graphs_3[0]

In [3]:
from collections import deque
import torch

def bfs_hop_distances(g, start_node):
    """
    Compute the shortest hop distance from start_node to all other nodes using BFS.
    
    Parameters:
        g (dgl.DGLGraph): The graph
        start_node (int): The starting node index
    
    Returns:
        distances (list): distances[i] = hop count from start_node to node i
    """
    N = g.num_nodes()
    distances = [-1] * N
    distances[start_node] = 0
    queue = deque([start_node])
    
    # Assuming undirected or properly oriented edges. If directed, consider successors/predecessors as needed
    while queue:
        u = queue.popleft()
        # Get neighbors
        for v in g.successors(u).tolist():
            if distances[v] == -1:  # not visited
                distances[v] = distances[u] + 1
                queue.append(v)
    return distances

def radius_of_action_bfs(g, start_node, X, mp_nb):
    """
    Efficiently compute radius of action up to mp_nb hops using BFS distances.
    
    Parameters:
        g (dgl.DGLGraph): Input graph
        start_node (int): Starting node
        X (torch.Tensor): Node coordinates, shape (N, D)
        mp_nb (int): Number of hops
    
    Returns:
        float: The computed radius of action.
    """
    distances = bfs_hop_distances(g, start_node)
    # Filter nodes within mp_nb hops
    within_hops = [i for i, d in enumerate(distances) if d != -1 and d <= mp_nb]
    
    # Compute Euclidean distances
    start_pos = X[start_node]
    sub_pos = X[within_hops]
    diff = sub_pos - start_pos
    dist = np.sqrt((diff ** 2).sum(axis=1))
    radius = dist.max() if dist.max() > 0.0 else 0.0
    return radius


def radius_of_action_bfs_optimized(g, start_node, X, mp_nb):
    # Use DGL's BFS generator to get layers
    # This might be more efficient than a Python BFS loop.
    frontiers = dgl.traversal.bfs_nodes_generator(g, start_node)
    
    visited = set()
    layer_id = 0
    nodes_within_hops = []
    for frontier in frontiers:
        nodes_in_layer = frontier.tolist()
        visited.update(nodes_in_layer)
        if layer_id <= mp_nb:
            nodes_within_hops.extend(nodes_in_layer)
        else:
            break
        layer_id += 1

    # Compute Euclidean distances
    start_pos = X[start_node]
    sub_pos = X[nodes_within_hops]
    diff = sub_pos - start_pos
    dist = np.sqrt((diff ** 2).sum(axis=1))
    radius = dist.max() if dist.size > 0 else 0.0
    return radius

In [36]:
def calculate_mean_max_radius(graph,X, x_hops):
    total_radius = 0.0
    num_nodes = graph.num_nodes()
    for node_index in range(num_nodes):
            radius = radius_of_action_bfs_optimized(graph,node_index,X, x_hops)
            total_radius += radius
    mean_radius = total_radius / num_nodes
    return mean_radius 

def calculate_mean_std_max_radius(graph, X, x_hops):
    total_radius = 0.0
    radii = []
    num_nodes = graph.num_nodes()
    for node_index in range(num_nodes):
        radius = radius_of_action_bfs_optimized(graph, node_index, X, x_hops)
        radii.append(radius)
        total_radius += radius
    mean_radius = total_radius / num_nodes
    std_dev_radius = np.std(radii)  # Calculate standard deviation
    return mean_radius, std_dev_radius

def CMMR(graph_path,mesh_path,x_hops):
    graphs, _ = dgl.load_graphs(graph_path) 
    graph = graphs[0]
    res_mesh = TelemacFile(mesh_path)
    X,triangles = add_mesh_info(res_mesh)
    mean = calculate_mean_max_radius(graph,X, x_hops)
    return mean

def CMSMR(graph_path,mesh_path,x_hops):
    graphs, _ = dgl.load_graphs(graph_path) 
    graph = graphs[0]
    res_mesh = TelemacFile(mesh_path)
    X,triangles = add_mesh_info(res_mesh)
    mean,std = calculate_mean_std_max_radius(graph,X, x_hops)
    return mean,std

In [62]:
mesh_path = '/work/m24046/m24046mrcr/results_data_30min_35_70/maillage_3.slf'
graph_path = '/work/m24046/m24046mrcr/results_data_30min_35_70/Mesh_base.bin'
#graph_path = '/work/m24046/m24046mrcr/results_data_30min_35_70_maillagex2/Mesh2_base.bin'

In [ ]:
import time 
t1 = time.time()
print(CMSMR(graph_path,mesh_path,10))
t2 = time.time()

In [ ]:
print(t2-t1)